<img src="Images/airplane.jpeg" alt="Aircraft in Flight" width="800">


# Project Goal
This project aims to analyze aviation accident data to identify **low-risk aircraft models** that the company should consider purchasing for its new aviation venture.

## Overview
This analysis examines aviation accident data from 1962 to 2023, covering civil aviation incidents in the United States and international waters. By exploring accident trends, aircraft models and flight phases where incidents are most frequent,the company can make informed decisions on acquiring aircraft that present the least operational risk.

The insights from this project will guide the company in selecting the safest and most reliable aircraft, ensuring smooth business operations and mitigating potential risks.

## 1. Business Problem

As the company expands into the aviation industry, it seeks to invest in aircraft for both commercial and private use. However, with limited knowledge of aircraft-related risks, it needs data-driven insights to determine which models are the safest and most suitable for its operations.

This analysis focuses on identifying low-risk airplanes by addressing the following key questions:

1️⃣ How are accidents distributed across different aircraft models?(*Which aircraft models have the lowest accident rates?*)

2️⃣ How reliable are different aircraft models over time? (*Are certain aircraft models consistently involved in accidents over multiple years?*)

3️⃣ How severe are accidents across different aircraft models?(*Which models have the least severe accidents?*)

By answering these questions, the company will gain crucial insights into aircraft safety, reliability, and operational risks. This information will help minimize potential liabilities, reduce operational disruptions, and protect the company’s reputation as it enters the aviation market.

## 2. Data Understanding
The data source of this analysis is from the CSV file 'Aviation_Data.csv' pulled from NTSB that includes aviation accident data from 1962 to 2023 about civil aviation accidents and selected incidents in the United States and international waters.

This dataset contains records(rows) that represents each aircraft accident event over the period of time.

The features total to 31, showing the details of each aircraft event such as the date of the accident, model and make of the aircraft, type of aircraft, injury severity and fatalities, weather conditions e.t.c

The target variables for this analysis will be the aircraft model in relation to the number of flight accidents, the flight phases for the accidents, the severity of the accidents in relation to the aircrafte make and model.

The data requires cleaning which shall be handled appropriately making it viable for use in our analysis.

In [ ]:
#Import necessary packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
#Loading dataset
df = pd.read_csv('Aviation_Data.csv', encoding = 'latin-1')

#Accessing the first 5 rows of the dataframe
df.head()

In [ ]:
#Dataframe Summary
df.info()

### Converting the dates column to datetime type

In [ ]:
#Converting Event.Date column to datetime format
df['Event.Date'] = pd.to_datetime(df['Event.Date'], format = '%Y-%m-%d')
#Converting Publication.Date column to datetime format
df['Publication.Date'] = pd.to_datetime(df['Publication.Date'], format = '%d-%m-%Y')

In [ ]:
#Checking the data types of the columns
df.dtypes

## 3. Data Preparation

### Data Cleaning
To answer the questions for this analysis, the data requires cleaning of missing values by applying the various methods to drop, replace or keep the values.

First is identifying the amount of missing data in each column.

In [ ]:
#Identifying missing values for each column in relation to the entire dataset
df.isna().mean().sort_values(ascending = False)

### Dropping columns
The 'Schedule' column is missing 86% of its data, hence it is best to drop the entire column.

In [ ]:
#Dropping the schedule column from the dataframe
df = df.drop('Schedule', axis = 1)
#Checking for other remaining null values
df.isna().sum()

### Dropping rows
The missing data from the 'Model' and 'Make' column is significantly low hence we can drop the rows with null values without much affecting the data as these columns are ideal for our analysis

In [ ]:
#Dropping rows with missing values in 'Model' column
df.dropna(subset = ['Model'], inplace = True)
#Dropping rows with missing values in 'Make' column
df.dropna(subset = ['Make'], inplace = True)
#Asserting no missing values left in the 'Make' and 'Model' columns
df.isna().sum()

## Replacing Missing Values
The 'Amateur.Built' column consists of categorical data of two types of values hence best to replace it with the mode as there is a few missing data.

In [ ]:
#Replacing null values with the mode
df['Amateur.Built'].fillna(df['Amateur.Built'].mode()[0], inplace = True)
assert df['Amateur.Built'].isna().sum()==0

For continuous data the best option would be replacing the missing data with the mean of the column.

In [ ]:
#Replacing missing values with the mean to the nearest whole number
df['Number.of.Engines'].fillna(round(df['Number.of.Engines'].mean()), inplace = True)
df['Total.Fatal.Injuries'].fillna(round(df['Total.Fatal.Injuries'].mean()), inplace=True)
df['Total.Serious.Injuries'].fillna(round(df['Total.Serious.Injuries'].mean()), inplace = True)
df['Total.Minor.Injuries'].fillna(round(df['Total.Minor.Injuries'].mean()), inplace = True)
df['Total.Uninjured'].fillna(round(df['Total.Uninjured'].mean()), inplace = True)

#Check for remaining null values
df.isna().sum()

## Keep missing values
Since the remaining missing values are in categorical data and provide valuable insights for analysis. The best approach is to keep them by replacing null values with the string 'missing' to indicate the absence of data.

In [ ]:
#Replacing the rest of the null values in the dataframe with the string missing
df.fillna('missing', inplace=True)
df.isna().sum()

## Column Combination
Step 1 : Changing all Make and Models to uppercase to avoid duplicate values due to format

In [ ]:
#Function that changes values to uppercase
def uppercase_strings(x):
    if isinstance(x, str):
        return x.upper()
    return x

#Apply lowercase_strings function to the columns
df[['Make', 'Model']] = df[['Make','Model']].applymap(uppercase_strings)
df[['Make', 'Model']]

Step 2: Combining the two columns to make one

In [ ]:
df['Make_Model'] = df ['Make'] + ',' + df['Model']
df['Make_Model']

## 4. Data Analysis

### a.) Accident Distribution by Make_Model
To address the first key question:

*How are accidents distributed across different aircraft models? (**Which aircraft models have the lowest accident rates?**)*

First we start by filtering the dataset to focus on airplanes and calculating the number of accidents recorded for each Make_Model. This helps in identifying patterns and trends in accident occurences across different aircraft models. 

Finally we will determine which models have the **lowest accident rates**, providing valuable insights into the safest options for the company's investment. 

In [ ]:
#Identifying rows whose aircraft category is airplane
Airplane_df = df[df['Aircraft.Category'] == 'Airplane']
Airplane_df

In [ ]:
#Counts of accidents per model
Airplane_df['Make_Model'].value_counts()

In [ ]:
#Plotting the bar plots 
fig, (ax1, ax2) = plt.subplots(ncols = 2, figsize = (16,6))

#Creating variables for easier reuse
top_30_counts = Airplane_df['Make_Model'].value_counts().iloc[:30]
bottom_5_counts = Airplane_df['Make_Model'].value_counts().iloc[-5:]

#Plot data using horizontal bar plots
ax1.barh(top_30_counts.index, top_30_counts.values, color = 'skyblue')
ax2.barh(bottom_5_counts.index, bottom_5_counts.values, color = 'lightcoral')

#Appearance
ax1.tick_params(axis = "y", labelsize = 10)
ax2.tick_params(axis = "y", labelsize = 10)
ax1.set_xlabel("Number of Aircraft Accidents")
ax2.set_xlabel("Number of Aircraft Accidents")
ax1.set_ylabel("Aircraft Make_Model")
ax2.set_ylabel("Aircraft Make_Model")
ax1.set_title("Top 30 Aircraft Make_Models by Total Accidents")
ax2.set_title("Last 5 Aircraft Make_Models by Total Accidents")

#Layout and Show plot
plt.tight_layout()
plt.show()

### b.) Aircraft Reliability Over Time
To address the second question:

*How reliable are different aircraft models over time? (**Are certain aircraft models consistently involved in accidents over multiple years?**)*

**Steps**

**1. Trend Analysis of Accidents by Aircraft Model:**
   The first step is to calculate the number of accidents per aircraft model for each year to identify patterns. This helps identify whether certain models show a recurring pattern of accidents or if incidents are isolated events.

**2. Identifying High-Risk vs. Low-Risk Trends:**
   The next step is to determine which aircraft models consistently appear in accident records over multiple years. Additionally, we will highlight models that have shown improved safety by reducing accident occurrences over time.

    

In [ ]:
# Trend Analysis of Accidents by Aircraft Model
df['Year'] = df['Event.Date'].dt.year  # Extract year from event date

# Grouping by Make_Model and Year to count accidents
accidents_per_year = df.groupby(['Make_Model', 'Year']).size().unstack(fill_value=0)

# Plotting trends for the top 10 aircraft models with the most accidents
top_10_models = accidents_per_year.sum(axis=1).nlargest(10).index
accidents_per_year.loc[top_10_models].T.plot(figsize=(12,6), marker='o')
plt.title("Accident Trends for Top 10 Aircraft Models Over Time")
plt.xlabel("Year")
plt.ylabel("Number of Accidents")
plt.legend(title="Aircraft Model")
plt.grid()
plt.show()


In [ ]:
# Trend Analysis for Least Accident-Prone Models
least_10_models = accidents_per_year.sum(axis=1).nsmallest(10).index
accidents_per_year.loc[least_10_models].T.plot(figsize=(12,6), marker='o')

plt.title("Accident Trends for 10 Least Accident-Prone Aircraft Models Over Time")
plt.xlabel("Year")
plt.ylabel("Number of Accidents")
plt.legend(title="Aircraft Model")
plt.grid()
plt.show()

In [ ]:
# 2. Identifying High-Risk vs. Low-Risk Trends
# High-risk: Models appearing in accidents for at least 5 different years
high_risk_models = accidents_per_year[accidents_per_year.gt(0).sum(axis=1) >= 5]
print("High-Risk Aircraft Models (Involved in accidents across 5+ years):")
print(high_risk_models.index.tolist())

# Low-risk: Models appearing in accidents for only 1 year
low_risk_models = accidents_per_year[accidents_per_year.gt(0).sum(axis=1) == 1]
print("Low-Risk Aircraft Models (Accidents recorded in only 1 year):")
print(low_risk_models.index.tolist())

## c.) Accident Severity by Model

To answer the question:

*How severe are accidents across different aircraft models?(**Which models have the least severe accidents?**)*

**Steps:**
1. **Categorizing Accident Severity by Aircraft Model:**
   
   The first step is to assess accident severity using the available "Injury.Severity" column. This column  classifies accidents as Fatal, Serious, Minor, or None. We will categorize aircraft models based on these classifications.

2. **Identifying Models with High and Low Severity Accidents:**
   
   To evaluate accident severity per model, we will:

      Determine aircraft models with the highest proportion of fatal accidents (most severe).

      Identify aircraft models with the lowest proportion of fatal accidents (least severe).

      Compare different aircraft models based on the total number of fatal, serious, minor, and uninjured incidents.




In [ ]:
# Aggregating injury data per aircraft model
agg_data = Airplane_df.groupby('Make_Model').agg({
    'Total.Fatal.Injuries': 'sum',
    'Total.Serious.Injuries': 'sum',
    'Total.Minor.Injuries': 'sum',
    'Total.Uninjured': 'sum'
})
agg_data

In [ ]:
# Calculating total injuries for sorting
agg_data['Total.Injuries'] = agg_data['Total.Fatal.Injuries'] + agg_data['Total.Serious.Injuries'] + agg_data['Total.Minor.Injuries']

# Sorting the data by total injuries
top_20 = agg_data.sort_values(by='Total.Injuries', ascending=False).head(20)
bottom_5 = agg_data.sort_values(by='Total.Injuries', ascending=True).head(5)  # Selecting the bottom 5 explicitly

# Creating the plots
fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(18, 8))

# Top 20 aircraft models with the most injuries
top_20[['Total.Fatal.Injuries', 'Total.Serious.Injuries', 'Total.Minor.Injuries', 'Total.Uninjured']].plot(
    kind='bar', stacked=True, ax=ax1, 
    color=['red', 'darkorange', 'gold', 'lightgreen'], alpha=0.85
)
ax1.set_title('Top 20 Aircraft Models by Total Injuries')
ax1.set_ylabel('Total Count')
ax1.set_xlabel('Aircraft Make_Model')
ax1.tick_params(axis="x", labelrotation=90)

# Bottom 5 aircraft models with the least injuries
bottom_5[['Total.Fatal.Injuries', 'Total.Serious.Injuries', 'Total.Minor.Injuries', 'Total.Uninjured']].plot(
    kind='bar', stacked=True, ax=ax2, 
    color=['red', 'darkorange', 'gold', 'lightgreen'], alpha=0.85
)
ax2.set_title('Bottom 5 Aircraft Models by Total Injuries')
ax2.set_ylabel('Total Count')
ax2.set_xlabel('Aircraft Make_Model')
ax2.tick_params(axis="x", labelrotation=90)

# Adjusting legend placement
handles, labels = ax1.get_legend_handles_labels()
fig.legend(handles, labels, title='Injury Type', bbox_to_anchor=(1.02, 1), loc='upper left')

#Layout and show plot
plt.tight_layout()
plt.show()

In [ ]:
df.to_csv("cleaned_data.csv", index=False)

## 5. Conclusion
This analysis provides valuable insights into aviation accident trends, enabling the company to make informed decisions about selecting the safest aircraft models for its new aviation division. Based on the findings, three key recommendations emerge:

**1. Selecting Aircraft Models with the Lowest Accident Occurrence**

The analysis reveals that different aircraft models have varying accident rates. To enhance operational safety, the company should prioritize acquiring aircraft models with the lowest accident records.

The top five safest models based on accident frequency include:

- MILLER ROGER, THATCHER CX4

- AVIAT AIRCRAFT INC, HUSKY

- PIPER, J3C-75

- WACO, YMF-3

- DIAMOND AIRCRAFT INDUSTRIES INC, DA 20-C1

**2. Selecting Aircraft Models with the Most Reliable Safety Records Over Time**

The analysis of accident trends over multiple years reveals that some aircraft models consistently experience more accidents, while others demonstrate greater reliability. The key insights include:

Certain models, such as CESSNA_152 and CESSNA_172, show a significant spike in accident occurrences at certain periods, indicating potential operational risks.

Other models, however, maintain relatively lower and stable accident rates over time, making them more suitable for selection.

The safest aircraft models based on historical accident trends include:

- CESSNA_150M

- CESSNA_180

- CESSNA_182

- ZENAIR, CH 2000

- YOUKEY, LANCAIR 235

These models have demonstrated low and stable accident rates over multiple years, making them ideal candidates for the company’s new aviation division. Prioritizing these models can enhance safety and reduce operational risks.

**3. Selecting Aircraft Models with the Lowest Fatalities and Highest Survival Rates**

The safest aircraft models should not only have fewer accidents but also record lower fatality and injury rates. Aircraft that minimize injuries and maximize passenger safety will enhance the company’s reputation and attract customers.

The best aircraft models based on survival rates include:

- GULFSTREAM, GULFSTREAM G150

- GULFSTREAM, GULFSTREAM GVI

- GULFSTREAM, GULFSTREAM150

- GULFSTREAM, GULFSTREAM200

- ZWICKER MURRAY R, GLASTAR

By focusing on these aircraft models and ensuring they are equipped with modern safety features, the company can improve operational efficiency and minimize aviation risks.

## 6. Further Analysis
Additional research can provide deeper insights into accident patterns and aircraft safety, further refining the company's selection strategy. The following areas warrant further investigation:

*1. Predicting Accidents Based on Weather Conditions*

Analyzing how weather variables (e.g., storms, fog, wind speed) influence accident rates could help develop a risk mitigation strategy for different flight conditions.

*2. Modeling Accident Frequency by Location*

Identifying high-risk locations based on accident data can help plan safer routes and avoid accident-prone airspaces.

*3. Assessing the Impact of Aircraft Maintenance on Accident Rates*

Investigating how maintenance frequency and quality affect accident occurrence will allow the company to establish best practices for fleet maintenance.

By conducting these additional analyses, the company can develop a comprehensive risk management strategy and optimize safety standards in its aviation division.